# Query Checking in Declare4Py

This tutorial explains how to perform the query checking of a DECLARE constraint in a log and how to browse the results.

After importing the Declare4Py package, a `Declare4Py` object has to be instantiated to load the log.

In [1]:
import sys
sys.path.append("..")
import os
from src.api.declare4py import Declare4Py


log_path = os.path.join("..", "test", "Sepsis Cases.xes.gz")

d4py = Declare4Py()
d4py.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The query checking is performed with the `query_checking` function. This takes as input the boolean parameter `consider_vacuity=true` that means that vacuously satisfied traces are considered as satisfied, violated otherwise. The parameter `template_str` sets the DECLARE constraint. Then the string parameters `activation` and `target` sets the variables to ask. If these parameters are not set then they are considered as variables to which compute the assignments. The input string parameters `act_cond`, `trg_cond`, `time_cond` are used to set the activation, target and time conditions (if needed), respectively. The float `min_support` parameter sets the support to be satisfied in the log by the variable assignments.

Two related query checking tasks can be performed by setting the `return_first` boolean parameter:
1. `return_first=False` returns all the variables assignments that satisfy the support in the log.
2. `return_first=True` returns only one variables assignment (the first) that satisfy the support in the log. This saves computational time when one is only interested in the existence of a variable assignment with a given support.

The returned data structure is a Python dictionary with keys the Declare constraints satisfying the assignments. The values are a structured representations of these constraints. Let's compute the assignments for the target by setting `template_str='Chain Response'`, `activation='IV Antibiotics'`, `act_cond='A.org:group is A'` and `min_support=0.2`:

In [2]:
res_1 = d4py.query_checking(consider_vacuity=True, template_str='Chain Response', activation='IV Antibiotics', act_cond='A.org:group is A', min_support=0.2, return_first=False)

# exploring the results
for k, v in res_1.items():
    print(f"{k} -> {v}")

Computing query checking ...
Chain Response[IV Antibiotics, ER Triage] |A.org:group is A | | -> {'template': 'Chain Response', 'activation': 'IV Antibiotics', 'target': 'ER Triage', 'act_cond': 'A.org:group is A', 'trg_cond': '', 'time_cond': ''}
Chain Response[IV Antibiotics, Release E] |A.org:group is A | | -> {'template': 'Chain Response', 'activation': 'IV Antibiotics', 'target': 'Release E', 'act_cond': 'A.org:group is A', 'trg_cond': '', 'time_cond': ''}
Chain Response[IV Antibiotics, LacticAcid] |A.org:group is A | | -> {'template': 'Chain Response', 'activation': 'IV Antibiotics', 'target': 'LacticAcid', 'act_cond': 'A.org:group is A', 'trg_cond': '', 'time_cond': ''}
Chain Response[IV Antibiotics, IV Liquid] |A.org:group is A | | -> {'template': 'Chain Response', 'activation': 'IV Antibiotics', 'target': 'IV Liquid', 'act_cond': 'A.org:group is A', 'trg_cond': '', 'time_cond': ''}
Chain Response[IV Antibiotics, Admission IC] |A.org:group is A | | -> {'template': 'Chain Respons

The query checking results can be filtered with the `filter_query_checking()` function whose parameter `queries` takes as input a list of variables to ask. In the above example, we set the template and activation and ask for the targets. We now filter the results by returning only the targets:

In [3]:
for tg in d4py.filter_query_checking(queries=['target']):
    print(tg)

['ER Triage']
['Release E']
['LacticAcid']
['IV Liquid']
['Admission IC']
['Release B']
['Release D']
['CRP']
['Return ER']
['Leucocytes']
['ER Sepsis Triage']
['Release A']
['Release C']
['Admission NC']
['ER Registration']


In the following example we compute the assignments both for the activation and the target by setting `template_str='Response'` and `min_support=0.8`:

In [4]:
res_2 = d4py.query_checking(consider_vacuity=False, template_str='Response', min_support=0.8, return_first=False)

# exploring the results
for k, v in res_2.items():
    print(f"{k} -> {v}")

Computing query checking ...
Response[ER Triage, CRP] | | | -> {'template': 'Response', 'activation': 'ER Triage', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Triage, Leucocytes] | | | -> {'template': 'Response', 'activation': 'ER Triage', 'target': 'Leucocytes', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Triage, ER Sepsis Triage] | | | -> {'template': 'Response', 'activation': 'ER Triage', 'target': 'ER Sepsis Triage', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Sepsis Triage, CRP] | | | -> {'template': 'Response', 'activation': 'ER Sepsis Triage', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Sepsis Triage, Leucocytes] | | | -> {'template': 'Response', 'activation': 'ER Sepsis Triage', 'target': 'Leucocytes', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Registration, ER Triage] | | | -> {'template': 'Response', 'activation': 'ER Registration', 'target': 'ER Triage', 'a

In this example, we set the template and ask for the activations and targets. We now filter the results by returning both activations and targets:

In [5]:
for tg in d4py.filter_query_checking(queries=['activation', 'target']):
    print(tg)

['ER Triage', 'CRP']
['ER Triage', 'Leucocytes']
['ER Triage', 'ER Sepsis Triage']
['ER Sepsis Triage', 'CRP']
['ER Sepsis Triage', 'Leucocytes']
['ER Registration', 'ER Triage']
['ER Registration', 'LacticAcid']
['ER Registration', 'CRP']
['ER Registration', 'Leucocytes']
['ER Registration', 'ER Sepsis Triage']


In [6]:
res_3 = d4py.query_checking(consider_vacuity=False, activation='ER Registration', target='CRP', min_support=0.2, return_first=False)

Computing query checking ...


In [7]:
for k, v in res_3.items():
    print(f"{k} -> {v}")

Choice[ER Registration, CRP] | | | -> {'template': 'Choice', 'activation': 'ER Registration', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Responded Existence[ER Registration, CRP] | | | -> {'template': 'Responded Existence', 'activation': 'ER Registration', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Response[ER Registration, CRP] | | | -> {'template': 'Response', 'activation': 'ER Registration', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Alternate Response[ER Registration, CRP] | | | -> {'template': 'Alternate Response', 'activation': 'ER Registration', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Precedence[ER Registration, CRP] | | | -> {'template': 'Precedence', 'activation': 'ER Registration', 'target': 'CRP', 'act_cond': '', 'trg_cond': '', 'time_cond': ''}
Alternate Precedence[ER Registration, CRP] | | | -> {'template': 'Alternate Precedence', 'activation': 'ER Registration', 'target': 'CRP',